In [1]:
import nltk
nltk.download('punkt')

# from google.colab import drive
# drive.mount('/content/drive')

import pandas as pd
import numpy as np
import codecs
import math
import nltk
import re 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import random
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
MAX_LENGTH=30
EOS_token=1
SOS_token=0

# ID2Vec_eng={}
# word2ID_eng={}
# ID2word_eng={}

# ID2Vec_hi={}
# word2ID_hi={}
# ID2word_hi={}

# combo_train = []
# combo_val = []
# combo_test = []

with open('/content/drive/MyDrive/Deep Learning/word2ID_eng.pkl', 'rb') as f:
        word2ID_eng = pickle.load(f)
with open('/content/drive/MyDrive/Deep Learning/ID2word_eng.pkl', 'rb') as f:
        ID2word_eng = pickle.load(f)
with open('/content/drive/MyDrive/Deep Learning/word2ID_hi.pkl', 'rb') as f:
        word2ID_hi = pickle.load(f)
with open('/content/drive/MyDrive/Deep Learning/ID2word_hi.pkl', 'rb') as f:
        ID2word_hi = pickle.load(f)
        
with open('/content/drive/MyDrive/Deep Learning/combo_train.pkl', 'rb') as f:
        combo_train= pickle.load(f)
with open('/content/drive/MyDrive/Deep Learning/combo_val.pkl', 'rb') as f:
        combo_val= pickle.load(f)
with open('/content/drive/MyDrive/Deep Learning/combo_test.pkl', 'rb') as f:
        combo_test= pickle.load(f)

# mispell=[]

with open('/content/drive/MyDrive/Deep Learning/mispell.pkl', 'rb') as f:
        mispell= pickle.load(f)

# eng_data=np.array()
# hin_data=np.array()

with open('/content/drive/MyDrive/Deep Learning/eng_data.pkl', 'rb') as f:
        eng_data= pickle.load(f)
with open('/content/drive/MyDrive/Deep Learning/hin_data.pkl', 'rb') as f:
        hin_data= pickle.load(f)
        

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
 def sentenceToIDs_eng(sentence):
    list=[]
    for token in sentence.split(' '):
        if token in mispell:
            list.append(word2ID_eng['a'])
        elif token not in word2ID_eng.keys():
          list.append(word2ID_eng['UNK'])
        else:
            list.append(word2ID_eng[token])
    list.append(word2ID_eng['EOS'])  
    return list

def sentenceToIDs_hi(sentence):
    list= []
    for token in sentence.split(' '):
        if token not in word2ID_hi.keys():
            list.append(word2ID_hi['UNK'])
        else:
            list.append(word2ID_hi[token])
    list.append(word2ID_hi['EOS'])  
    return list
        

In [5]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [6]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        #self.embedding = lambda x: torch.FloatTensor(list(ID2Vec_eng[x.item()]))
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [7]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [8]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]


    decoder_input = torch.tensor([[0]], dtype=torch.long, device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == 1:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [9]:
def trainIters(encoder, decoder, n_iters, instancestrained, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        id=random.randint(0+instancestrained, 100000-1-2000+instancestrained)
        input_tensor = torch.tensor(hin_data[id], dtype=torch.long, device=device).view(-1, 1)
        if(input_tensor.size()[0]>MAX_LENGTH):
            continue
        target_tensor = torch.tensor(eng_data[id], dtype=torch.long, device=device).view(-1, 1)

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [10]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [11]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = torch.tensor(sentenceToIDs_hi(sentence), dtype=torch.long, device=device).view(-1, 1)
        #input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(ID2word_eng[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [12]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        randId = random.randint(0, 100000-1)
        if(len(combo_train[randId][0].split(' '))>=MAX_LENGTH):
            continue
        print('>', combo_train[randId][0])
        print('=', combo_train[randId][1])
        output_words, attentions = evaluate(encoder, decoder, combo_train[randId][0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [13]:
hidden_size = 256
input_size=len(word2ID_hi)
output_size=len(word2ID_eng)

encoder1 = EncoderRNN(input_size, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_size, dropout_p=0.1).to(device)

previoustrained = 0
iter= 80000 #len(combo_train)

# encoder1.load_state_dict(torch.load('encoder_new_h2e_Run3.dict'))
# attn_decoder1.load_state_dict(torch.load('decoder_new_h2e_Run3.dict'))
trainIters(encoder1, attn_decoder1, iter, previoustrained,print_every=10000)


NameError: ignored

In [ ]:
torch.save(encoder1.state_dict(), 'encoder_new_h2e_Run4.dict')
torch.save(attn_decoder1.state_dict(), 'decoder_new_h2e_Run4.dict')

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)


> साइरिलिक रूसी
= cyrillic russian
< and UNK UNK <EOS>

> भाग
= function
< UNK <EOS>

> क्या प्रतीक पर संदेश दिखाना है जब नया संदेश प्रकट होता है
= whether show message over the icon when new messages arrive
< do they make the UNK of the UNK UNK <EOS>

> s नहीं बचाया जा सकाः s
= could not save a text part s
< no UNK UNK the UNK UNK <EOS>

> एक खाली खाँचाstockslot hint
= s on reserve
< and UNK UNK UNK <EOS>

> पसंद निर्यात करें E …
= export bookmarks…
< export UNK UNK <EOS>

> जारी रखें वर्तमान काल के संदेश का सम्पादन
= continue editing current message
< and the UNK the the UNK the <EOS>

> सुरक्षा
= security
< security <EOS>

> फिल्टरों को व्यवस्थित करें
= f ilters
< and UNK UNK of <EOS>



**Testing our model**

In [ ]:
import torch
from torchtext.data.metrics import bleu_score


def evaluate_testData(encoder, decoder, test_data):
    for i in range(len(test_data)):
        if(len(test_data[i][0].split(' '))>=MAX_LENGTH):
          continue
        references_corpus.append([test_data[i][1].split(' ')])
        output_words, attentions = evaluate(encoder, decoder, test_data[i][0])
        candidate_corpus.append(output_words)
        output_sentence = ' '.join(output_words)

**BLEU Score on test data**

In [ ]:
candidate_corpus = []
references_corpus =[]
evaluate_testData(encoder1, attn_decoder1, combo_test)
print("bleu_score on test data: " +str(bleu_score(candidate_corpus, references_corpus)))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.022940624931838245

**BLEU score on train data**

In [ ]:
candidate_corpus = []
references_corpus =[]
evaluate_testData(encoder1, attn_decoder1, combo_train[0:1000])
print("bleu_score on train data: " +str(bleu_score(candidate_corpus, references_corpus)))

0.2684824121286717

**BaseLine Implementation**

In [ ]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 1.3 MB 10.7 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 65 kB 3.2 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=2dc93c7eaf2f38b098e27417b673e36dcea8f54c82b468dc56fd3b41b1fe06c9
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [ ]:
#!pip install googletrans
import googletrans
from googletrans import Translator
translator = Translator()

In [ ]:
def evaluate_Baseline_TranslateRandom(test_data):
    for i in range(len(test_data)):
        if(len(test_data[i][0].split(' '))>=MAX_LENGTH):
          continue
        ref=test_data[i][1].split(' ')
        cand=[]
        for word in test_data[i][0].split(' '):
          #cand.append(random.choice(list(word2ID_eng.keys())))
          cand.append(random.choice(['a']))
        print('>', test_data[i][0])
        print('=', test_data[i][1])
        output_sentence = ' '.join(cand)
        print('<', output_sentence)
        print('')
        references_corpus.append([ref])     
        candidate_corpus.append(cand)

**BLEU Score of Baseline on test data**

In [ ]:
candidate_corpus = []
references_corpus =[]
evaluate_Baseline_TranslateRandom(combo_test)
nltk.translate.bleu_score.corpus_bleu(references_corpus, candidate_corpus)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.3910044809521771